TRABALHO BD

importar um registo e guardar uma string locus
Ir buscar o ficheiro GenBank pelo webscrapping

A partir daqui ir buscar locus, id, definition, accession... pelas expressões regulares

Colocar as infos na BD

Escolha de sequências
-> escolhemos 1 vírus (HIV) e uma bactéria (H. pylori)
-> motivos das escolhas: sequências pequenas (teoricamente) e mais fácil de analisar

WEBSCRAPPING

In [2]:
import time
query = input("Defina o que pretende pesquisar [bacteria/yeast/...]: ")
def url_get(item):
    url_list = []
    url = "https://www.ncbi.nlm.nih.gov/gene/?term={}".format(query)
    url_list.append(url)
    return url_list
    
url_get(0)

['https://www.ncbi.nlm.nih.gov/gene/?term=yeast']

In [3]:
import requests, sys, json
content = []
for url in url_get(0):
    r = requests.get(url)
    content.append(r.content)
# print(content)

In [4]:
import requests
from bs4 import BeautifulSoup

# Parsing the HTML
for c in content:
    soup = BeautifulSoup(r.content, 'html.parser')
    res = soup.get_text()
# print(res)

In [5]:
import re
existe = re.findall(r'ID:\s+\d*(?=\D)', res, re.DOTALL)
id_genes = ','.join(existe)
id_g = id_genes.replace('ID: ', '')
id_gene = id_g.split(',')
id_gene


['424122',
 '29789',
 '26245415',
 '20918',
 '24563533',
 '39874648',
 '4839978',
 '42049400',
 '42048491',
 '36516274',
 '35606568',
 '35598500',
 '24564880',
 '24564814',
 '13446414',
 '5477914',
 '65082716',
 '109860985',
 '8047674',
 '4292']

In [6]:
number_of_genes = int(input("Número de genes a obter (máx: 20): "))
print(number_of_genes)
genes = id_gene[0:number_of_genes]
print(genes)

5
['424122', '29789', '26245415', '20918', '24563533']


In [ ]:
#BUSCAR LINKS DOS GENES

In [7]:
import time
def url_get_id(item):
    url_id = []
    for id in genes:
        url = f"https://www.ncbi.nlm.nih.gov/nuccore/{id}"
        url_id.append(url)
    return url_id
url_get_id(genes)

['https://www.ncbi.nlm.nih.gov/nuccore/424122',
 'https://www.ncbi.nlm.nih.gov/nuccore/29789',
 'https://www.ncbi.nlm.nih.gov/nuccore/26245415',
 'https://www.ncbi.nlm.nih.gov/nuccore/20918',
 'https://www.ncbi.nlm.nih.gov/nuccore/24563533']

In [8]:
import requests
id_content = []
for url in url_get_id(genes):
    r = requests.get(url)
    id_content.append(r.content)
# id_content

In [93]:
import requests
from bs4 import BeautifulSoup

# Making a GET request
r = requests.get('https://www.ncbi.nlm.nih.gov/nuccore/{}'.format(genes))
# Parsing the HTML
locus = ""
for c in id_content:
	soup = BeautifulSoup(c, 'html.parser')

# Procurar um tag meta com um determinado atributo

	lines = soup.find_all('meta', {'name':"ncbi_uidlist"} )

	id = ""
	url = ""
	for line in lines:
		# print(line)
		# if 'name' in line.attrs:
		# 	print(line.attrs['name'])
		if 'content' in line.attrs:
			# print(line.attrs['content'])		
			id = line.attrs['content']

	if id:
		url ="https://www.ncbi.nlm.nih.gov/sviewer/viewer.fcgi?id={}&db=nuccore&report=genbank&conwithfeat=on&hide-cdd=on&retmode=text&ncbi_phid=CE88F25338A215A1000000000483042A&withmarkup=on&tool=portal&log$=seqview&maxdownloadsize=1000000".format(id)

	r2 = requests.get(url)
	locus += r2.content.decode('utf-8')
	print(locus)
	#utf-8 para converter de bytes para string
	#https://stackoverflow.com/questions/31019854/typeerror-cant-use-a-string-pattern-on-a-bytes-like-object-in-re-findall

LOCUS       CSU02075                1780 bp    rRNA    linear   INV 28-JUN-1994
DEFINITION  Chlorarachnion sp. 1 CCMP 1408 clones pF58 and pF59 18S rRNA small
            subunit.
ACCESSION   U02075
VERSION     U02075
KEYWORDS    .
SOURCE      Chlorarachnion sp. 1
  ORGANISM  Chlorarachnion sp. 1
            Eukaryotae; mitochondrial eukaryotes; Chlorarachniophyceae;
            Chlorarachnion.
REFERENCE   1  (bases 1 to 1780)
  AUTHORS   McFadden,G.I., Gilson,P.R., Hofmann,C.J., Adcock,G.J. and
            Maier,U.G.
  TITLE     Evidence that an amoeba acquired a chloroplast by retaining part of
            an engulfed eukaryotic alga
  JOURNAL   Proc. Natl. Acad. Sci. U.S.A. 91, 3690-3694 (1994)
  MEDLINE   94224807
REFERENCE   2  (bases 1 to 1780)
  AUTHORS   McFadden,G.I.
  TITLE     Direct Submission
  JOURNAL   Submitted (15-NOV-1993) Geoffrey I. McFadden, School of Botany,
            University of Melbourne, Parkville, VIC 3052, Australia
COMMENT     [WARNING] On Jan 20, 1999 t

LINK NCBI

In [72]:
import re
id_lista = []
# print(genes)
existe = re.findall(r'ACCESSION\s+[^\s]+', locus)
#\s+ um ou mais espaços
#[^\s]+ indica um conjunto de caracteres que se podem repetir uma ou mais vezes

if existe:
    for c in existe:
        m = re.match(r'ACCESSION\s+([^\s]+)', c )
        if m:
            id = m.group(1)
            id_lista.append("https://www.ncbi.nlm.nih.gov/nuccore/{}".format( id ) )
else:
    print( "Padrão não encontrado" )
id_lista



['https://www.ncbi.nlm.nih.gov/nuccore/U02075',
 'https://www.ncbi.nlm.nih.gov/nuccore/X07874',
 'https://www.ncbi.nlm.nih.gov/nuccore/AY158036',
 'https://www.ncbi.nlm.nih.gov/nuccore/BW173609']

ACCESSION

In [75]:
acc_list = []

existe = re.findall(r'ACCESSION\s+.*?(?=VERSION)', locus, re.DOTALL)
#\s+ um ou mais espaços
#\.* encontra o caractere ponto final zero ou mais vezes
#?= something que faz match com VERSION mas não consome estes carateres
if existe:
    for accession in existe:
        m = re.match( r'ACCESSION\s+(.+)', accession, re.DOTALL )
        acc_list.append(re.sub(r'\s+', ' ', m.group(1) ) )
#(.+) agrupar um ou mais caracteres dentro de um grupo
#re.sub(r'\s+', ' ' ...) substitui um ou mais espaços por apenas um espaço
acc_list

['U02075 ', 'X07874 ', 'AY158036 ', 'BW173609 ']

DEFINITION

In [65]:
defi_list = []
existe = re.findall(r'DEFINITION\s+.*?(?=ACCESSION)', locus, re.DOTALL)
if existe:
    for definition in existe:
        m = re.match( r'DEFINITION\s+(.+)', definition, re.DOTALL )
        defi_list.append(re.sub(r'\s+', ' ', m.group(1) ) )
defi_list

['Chlorarachnion sp. 1 CCMP 1408 clones pF58 and pF59 18S rRNA small subunit. ',
 "Human CD2 gene exon 5 and 3'-flanking region. ",
 'Gamma proteobacterium UQ1 16S ribosomal RNA gene, partial sequence. ',
 "BW173609 Nori Satoh unpublished cDNA library, neural complex Ciona intestinalis cDNA clone rcinc027k05 3', mRNA sequence. "]

LOCUS

In [77]:
loc_list = []
existe = re.findall(r'LOCUS\s+.*?(?=DEFINITION)', locus, re.DOTALL)
if existe:
    for loc in existe:
        m = re.match( r'LOCUS\s+(.+)', loc, re.DOTALL )
        loc_list.append(re.sub(r'\s+', ' ', m.group(1) ) )
loc_list

['CSU02075 1780 bp rRNA linear INV 28-JUN-1994 ',
 'HSCD2G5 807 bp DNA linear PRI 30-SEP-1993 ',
 'AY158036 502 bp DNA linear BCT 08-DEC-2002 ',
 'BW173609 778 bp mRNA linear EST 07-JAN-2011 ']

LOCUS NAME

In [78]:
loc_name_list = []
existe = re.findall(r'LOCUS\s + [^\s]+', locus, re.DOTALL)
if existe:
    for id in existe:
        m = re.match( r'LOCUS\s+(.+)', id, re.DOTALL )
        loc_name_list.append(re.sub(r'\s+', ' ', m.group(1) ) )
loc_name_list

['CSU02075', 'HSCD2G5', 'AY158036', 'BW173609']

NUMBER OF BP

In [99]:
loc_bp_list = []
loc = []

existe = re.findall(r'LOCUS\s+.*?(?=DEFINITION)', locus, re.DOTALL)
# print(existe)
if existe:
    for c in existe:
        loc.append(list(filter(None, c.split(' '))))
for x in range(len(loc)):
    loc_bp_list.append(' '.join(loc[x][2:4])) 

#cria lista com os valores do locus, e escolhe os 3º e 4º caracteres, que correspondem ao nr de bp
loc_bp_list

['LOCUS       CSU02075                1780 bp    rRNA    linear   INV 28-JUN-1994\n', 'LOCUS       HSCD2G5                  807 bp    DNA     linear   PRI 30-SEP-1993\n', 'LOCUS       AY158036                 502 bp    DNA     linear   BCT 08-DEC-2002\n', 'LOCUS       BW173609                 778 bp    mRNA    linear   EST 07-JAN-2011\n']


['1780 bp', '807 bp', '502 bp', '778 bp']

MOLECULE TYPE

In [100]:
mol_type = []
for m in range(len(loc)):
    if "linear" in loc[m][5]:
        mol_type.append(' '.join(loc[m][4:6]))
    else:
        mol_type.append(' '.join(loc[m][4:5]))
#temos que corrigir esta porque as mol types podem ser de uma, duas ou trÊs palavras..
mol_type

['rRNA linear', 'DNA linear', 'DNA linear', 'mRNA linear']

GENBANK DIVISION

In [101]:
gb_div = []
for gb in range(len(loc)):
    gb_div.append(' '.join(loc[gb][6:7]))
gb_div

['INV', 'PRI', 'BCT', 'EST']

MODIFICATION DATE

In [104]:
mod_date = []
for md in range(len(loc)):
    mod_date.append(''.join(loc[md][7]).replace("\n",""))
mod_date

['28-JUN-1994', '30-SEP-1993', '08-DEC-2002', '07-JAN-2011']

SOURCE

In [106]:
source_list = []
existe = re.findall(r'SOURCE\s+.*?(?=ORGANISM)', locus, re.DOTALL)
# print(existe)
if existe:
    for source in existe:
        m = re.match( r'SOURCE\s+(.+)', source, re.DOTALL )
        source_list.append(re.sub(r'\s+', ' ', m.group(1) ) )
source_list

['SOURCE      Chlorarachnion sp. 1\n  ', 'SOURCE      Unknown.\n  ', 'SOURCE      gamma proteobacterium UQ1\n  ', 'SOURCE      Ciona intestinalis (vase tunicate)\n  ']


['Chlorarachnion sp. 1 ',
 'Unknown. ',
 'gamma proteobacterium UQ1 ',
 'Ciona intestinalis (vase tunicate) ']

ORGANISM

In [108]:
org_list = []
existe = re.findall(r'ORGANISM\s+.*?(?=\n)', locus, re.DOTALL)
if existe:
    for org in existe:
        m = re.match( r'ORGANISM\s+(.+)', org, re.DOTALL )
        org_list.append(re.sub(r'\s+', ' ', m.group(1) ) )
org_list

['Chlorarachnion sp. 1',
 'Unknown.',
 'gamma proteobacterium UQ1',
 'Ciona intestinalis']

FEATURES

In [ ]:
misc = []
existe = re.findall(r'FEATURES\s+.*?(?=ORIGIN)', locus, re.DOTALL)
#DOTALL ignora as linhas, pois as features ocupam várias linhas
if existe:
    for feat in existe:
        m = re.match( r'FEATURES\s+(.+)',feat, re.DOTALL )
        misc.append(re.sub(r'/',  ' ', m.group(1) ) )
    for c in misc:
        print(c, "\n")

FEATURES - SOURCE

In [ ]:

existe = re.findall(r'source\s+[1-9\.]*', locus, re.DOTALL)
if existe:
    for fsource in existe:
        print( re.sub(r'source\s+', ' ', fsource) )


FEATURES - EXONS

In [ ]:
existe = re.findall(r'exon\s+[1-9\>\.]*', locus, re.DOTALL)
if existe:
    for fexons in existe:
        print( re.sub(r'exon\s+', ' ', fexons) )

FEATURES - INTRON

In [ ]:
existe = re.findall(r'intron\s+[1-9\>\.]*', locus, re.DOTALL)
if existe:
    for fintrons in existe:
        print( re.sub(r'intron\s+', ' ', fintrons) )

CDS - JOIN

In [ ]:
existe = re.findall(r'CDS\s+?(join)[1-9\.\,\(\)]*', locus, re.DOTALL)
#(join) procura a palavra join 1x
#procura nrs de 1-9, ponto, virgula e parentises zero ou mais vezes
print(existe[0])
if existe:
    for cds in existe:
        print( re.sub(r'CDS\s+', ' ', cds) )

SEQUENCE/ORIGIN

In [109]:
seq_origin = []
existe = re.findall(r'ORIGIN\s+.*?(?=//)', locus, re.DOTALL)
if existe:
    for nuc in existe:
        m = re.match( r'ORIGIN\s+(.+)', nuc, re.DOTALL)
        a = re.sub(r'\s+', '', m.group(1) ) 
        num = re.sub(r'\d+','', a)
        seq_origin.append(num)
seq_origin

['ggatcctgccagtagtcatatgcttgtctcaaagactaagccatgcatgtctaagtataaacaactttgtactgtgaaactgcagatggctcattaaatcagaaataatttctttgatggtgaaaatctacacggatacccgtagtaattctagagctaatacgtgcgtaaactccattgtgaggtatttattagatccgaaccagccccctcggggtattttggtgaatcatgataactgttcgaatcgccaggctctgccaggcgatagttcattcaaatttctgccctatcagctattgacggtagtgtagtggactaccgtggccgtaacgggtgacggagaattagggttcgattccggagagggagcctgagaaacggctaccacatccaaggaaggcagcaggcgcgcaaattacccaatcccgacacggggaggtagtgacaagaaatatcgtgactgtgccctttgtggcagagtcttcgaaatgagcaatacgtaaatgcctttgcgagtatccattggagggcaagtctggtgccagcagccgcggtaattccagctccaatagcgtatattaatgttgttgcagttaaaaagctcgtagttggatatctggcctttgcacggacggcccgccttcgggttgagtgctggaagtgcaggtctatattctggttggcgtgctttccctttattgtggcgagtatcgcgatccggatcgtttactttgaagaaattcgagtgttcaaagcaggcttacgccgtgaagattgcagcatggaataatgacataggactatcgtgttctattttgttggttctgggaccgtagtaatgattgatagggacagttgggggtgctgctattccgcggncagaggtgaaattctcagattcccggaagagcaacttatgcgaaagcattcaccaaggatgtcttctttaatcaagaacgaaagtagggggatcgaagacgatcagataccgtcgt

In [140]:
seq_count = []
for seq in seq_origin:
    #contagem de nucleotidos
    A= []
    C= []
    T= []
    G= []
    for n in seq:
        if n == 'a':
            A.append('a')
        if n == 'c':
            C.append('c')
        if n == 't':
            T.append('t')
        if n == 'g':
            G.append('g')
        
    a = len(A)
    c = len(C)
    t = len(T)
    g = len(G)
    seq_count.append([a,c,t,g])
    #fazer append de um array
seq_count


[[457, 366, 484, 472],
 [204, 228, 196, 179],
 [130, 110, 99, 163],
 [259, 130, 209, 179]]

JOURNAL

In [138]:
journ_list = []
existe = re.findall(r'JOURNAL\s+.*?(?=PUBMED|REFERENCE|FEATURES)', locus, re.DOTALL)
if existe:
    for journal in existe:
        m = re.match( r'JOURNAL\s+(.+)', journal, re.DOTALL )
        journ_list.append(re.sub(r'\s+', ' ', m.group(1) ) )
journ_list
#| dá-nos operador ou


['Proc. Natl. Acad. Sci. U.S.A. 91, 3690-3694 (1994) MEDLINE 94224807 ',
 'Submitted (15-NOV-1993) Geoffrey I. McFadden, School of Botany, University of Melbourne, Parkville, VIC 3052, Australia COMMENT [WARNING] On Jan 20, 1999 this sequence was replaced by U02075.1. ',
 'EMBO J. 7 (6), 1675-1682 (1988) MEDLINE 89005055 COMMENT [WARNING] On Feb 18, 1996 this sequence was replaced by X07874.1. see x07871 - x07874 Data kindly reviewed (12-SEP-1988) by CRUMPTON J. ',
 'Unpublished ',
 'Submitted (02-OCT-2002) Molecular Biology, PROIMI, Av. Belgrano y Pje. Caseros, S.M. de Tucuman, Tucuman 4000, Argentina ',
 'Unpublished (2002) COMMENT Contact: Nori Satoh Department of Zoology Kyoto University Sakyo-ku, Kyoto, Kyoto 606-8502, Japan. ']

AUTHORS

In [141]:
authors_list = []
existe = re.findall(r'AUTHORS\s+.*?(?=TITLE)', locus, re.DOTALL)
if existe:
    for auth in existe:
        m = re.match( r'AUTHORS\s+(.+)', auth, re.DOTALL )
        authors_list.append(re.sub(r'\s+', ' ', m.group(1) ) )
authors_list

['McFadden,G.I., Gilson,P.R., Hofmann,C.J., Adcock,G.J. and Maier,U.G. ',
 'McFadden,G.I. ',
 'Lang,G., Wotton,D., Owen,M.J., Sewell,W.A., Brown,M.H., Mason,D.Y., Crumpton,M.J. and Kioussis,D. ',
 'Benito,J.M., Lovrich,G.A., Abate,C.M. and Sineriz,F. ',
 'Benito,J.M., Lovrich,G.A., Abate,C.M. and Sineriz,F. ',
 'Satou,Y., Shin-i,T., Kohara,Y. and Satoh,N. ']

TITLE

In [142]:
titles = []
existe = re.findall(r'TITLE\s+.*?(?=JOURNAL)', locus, re.DOTALL)
if existe:
    for title in existe:
        m = re.match( r'TITLE\s+(.+)', title, re.DOTALL )
        titles.append(re.sub(r'\s+', ' ', m.group(1) ) )
titles

['Evidence that an amoeba acquired a chloroplast by retaining part of an engulfed eukaryotic alga ',
 'Direct Submission ',
 'The structure of the human CD2 gene and its expression in transgenic mice ',
 'Microbial populations from the subantarctic marine ecosystem ',
 'Direct Submission ',
 'Expressed genes in Ciona intestinalis (2002c) ']

PUBMED

In [ ]:
existe = re.findall(r'PUBMED\s+.*?(?=REFERENCE)', locus, re.DOTALL)
if existe:
    for pubm in existe:
        m = re.match( r'PUBMED\s+(.+)', pubm, re.DOTALL )
        print( re.sub(r'\s+', ' ', m.group(1) ) )

AFFILIATION

In [ ]:
import requests
r = requests.get('https://pubmed.ncbi.nlm.nih.gov/2549509')
pub = r.content.decode('utf-8')
print(pub)

In [ ]:
import re
existe = re.search(r'affiliation-1\s*.*\.', pub)
#\s* zero ou mais espaços
#.* zero ou mais caracteres
#\. caractere ponto final -> procura affiliation até encontrar ponto final

if existe:
    m = re.findall(r'>\s.*', existe[0] )
    #o findall retorna uma lista, por isso temos que selecionar m[0] para fazer o replace
    m = m[0].replace("> ",'')
    print(m)

else:
    print( "Padrão não encontrado" )

AUTHOR NAME

In [ ]:
existe = re.search("citation_authors.*;", pub)
if existe:
    m = re.findall(r'=".*', existe[0] )
    #procura =" seguido de um ou mais caracteres
    m = m[0].replace('=\"','')
    #\" para indicar que queremos os caractere " (e não abrir aspas)
    print(m)